# Wersja v12: Model Zaawansowany (Top-K, Wagi, Analiza Błędów)

## Część 1: Przygotowanie Danych i Inżynieria Cech
**Nowość:** Przygotowanie danych pozostaje w większości bez zmian, ale zapewniamy, że wszystkie potrzebne informacje do ważenia próbek w Części 2 będą dostępne.

In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import gc
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

# Definicja stałych globalnych
ARTIFACTS_DIR = 'artifacts_v12'
PROCESSED_DATA_DIR = 'processed_data_v12'
MAX_TEXT_FEATURES = 20000
RANDOM_STATE = 42

# Tworzenie katalogów, jeśli nie istnieją
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

print(f"Katalogi '{ARTIFACTS_DIR}' i '{PROCESSED_DATA_DIR}' są gotowe.")

Katalogi 'artifacts_v12' i 'processed_data_v12' są gotowe.


In [2]:
def load_location_data(location_path='lokalizacja.csv'):
    print(f"Wczytywanie pliku: {location_path}")
    df_loc = pd.read_csv(
        location_path,
        na_values=['\\N', 'NULL'],
        sep=',',
        header=None,
        names=['id', 'parent_id', 'name', 'type', 'full_name']
    )
    return df_loc

def load_offers_data(offers_path='saleflats_2024_2025.csv'):
    print(f"Wczytywanie pliku: {offers_path}")
    column_names = ['title', 'description', 'area', 'price', 'locationPath']
    try:
        first_row = pd.read_csv(offers_path, header=None, sep=',', nrows=1, on_bad_lines='skip')
        last_col_index = first_row.shape[1] - 1
        usecols = [3, 4, 5, 6, last_col_index]
        df_offers = pd.read_csv(offers_path, header=None, sep=',', usecols=usecols, on_bad_lines='skip')
        df_offers.columns = column_names
    except Exception as e:
        print(f"Wystąpił błąd: {e}")
        df_offers = pd.DataFrame(columns=column_names)
    return df_offers

df_loc = load_location_data()
df_offers = load_offers_data()

# Stworzenie i zapisanie map
id_to_name = dict(zip(df_loc['id'], df_loc['name']))
joblib.dump(id_to_name, os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))
hierarchy_map = dict(zip(df_loc['id'], df_loc['parent_id']))
joblib.dump(hierarchy_map, os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
print("Mapy 'id_to_name' i 'hierarchy_map' zostały stworzone i zapisane.")

Wczytywanie pliku: lokalizacja.csv
Wczytywanie pliku: saleflats_2024_2025.csv
Mapy 'id_to_name' i 'hierarchy_map' zostały stworzone i zapisane.


In [3]:
# Cechy Numeryczne
df_offers['area'] = pd.to_numeric(df_offers['area'], errors='coerce')
df_offers['price'] = pd.to_numeric(df_offers['price'], errors='coerce')
df_offers.dropna(subset=['area', 'price'], inplace=True)
df_offers = df_offers[df_offers['area'] > 0]
df_offers['price_per_meter'] = df_offers['price'] / df_offers['area']

# Definicja celów: Dzielnica, Pod-dzielnica, Ulica
path_cols = ['woj_id', 'pow_id', 'gmi_id', 'city_id', 'district_id', 'subdistrict_id', 'street_id']
path_df = df_offers['locationPath'].str.split(',', expand=True)
path_df = path_df.iloc[:, :len(path_cols)]
path_df.columns = path_cols[:path_df.shape[1]]
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df_offers = pd.concat([df_offers.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)

# Finalne czyszczenie i tworzenie df_processed
df_offers.dropna(subset=['city_id'], inplace=True)
df_offers = df_offers[df_offers['city_id'] != 0]

df_processed = df_offers[[
    'title', 'description', 'area', 'price', 'price_per_meter', 'city_id', 
    'district_id', 'subdistrict_id', 'street_id'
]].copy()

df_processed.rename(columns={
    'district_id': 'target_district_id',
    'subdistrict_id': 'target_subdistrict_id',
    'street_id': 'target_street_id'
}, inplace=True)

df_processed['text_features'] = df_processed['title'].fillna('') + " " + df_processed['description'].fillna('')
df_processed.drop(columns=['title', 'description'], inplace=True)
df_processed.dropna(inplace=True)

print("\n--- Nagłówek finalnego df_processed ---\n")
print(df_processed.head())


--- Nagłówek finalnego df_processed ---

    area     price  price_per_meter  city_id  target_district_id  \
0  73.00  766500.0     10500.000000      352                   0   
1  64.80  540000.0      8333.333333      352                   0   
2  51.00  540000.0     10588.235294      352                   0   
3  67.62  544000.0      8044.957113      352                   0   
4  48.00  459000.0      9562.500000      352                   0   

   target_subdistrict_id  target_street_id  \
0                 103786                 0   
1                  99764                 0   
2                  74375                 0   
3                  74375            517513   
4                  95559                 0   

                                       text_features  
0  Mieszkanie trzypokojowe na sprzedaż Mieszkanie...  
1  Sprzedam mieszkanie na parterze 64.8m2 Białyst...  
2  Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki ...  
3  Mieszkanie trzypokojowe na sprzedaż ***Oferta ...

In [4]:
# Sprawdzenie liczności klas dla stratyfikacji
city_counts = df_processed['city_id'].value_counts()
valid_cities = city_counts[city_counts > 1].index
df_filtered = df_processed[df_processed['city_id'].isin(valid_cities)].copy()

# Podział na zbiory (X i y)
target_columns = ['target_district_id', 'target_subdistrict_id', 'target_street_id']
X = df_filtered.drop(columns=target_columns)
y = df_filtered[target_columns]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=X['city_id']
)
print(f"Podział danych: {len(X_train)} próbek treningowych, {len(X_val)} próbek walidacyjnych.")

# Przetwarzanie Cech Numerycznych
numeric_features = ['area', 'price', 'price_per_meter']
imputer = SimpleImputer(strategy='median'); scaler = StandardScaler()
X_train_num = scaler.fit_transform(imputer.fit_transform(X_train[numeric_features]))
X_val_num = scaler.transform(imputer.transform(X_val[numeric_features]))
joblib.dump(imputer, os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))

# Przetwarzanie Cech Tekstowych
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 2))
X_train_text = vectorizer.fit_transform(X_train['text_features'])
X_val_text = vectorizer.transform(X_val['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))

# Przygotowanie pozostałych cech
X_train_city = X_train['city_id'].values
X_val_city = X_val['city_id'].values

# Zapisanie y_train i y_val do plików .npy do użycia w Części 2
y_train.to_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_train.pkl'))
y_val.to_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_val.pkl'))

print("Transformatory dopasowane i zapisane.")

Podział danych: 887572 próbek treningowych, 221893 próbek walidacyjnych.
Transformatory dopasowane i zapisane.


In [5]:
# Zapisanie wszystkich przetworzonych danych
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'), X_train_text)
scipy.sparse.save_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'), X_val_text)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'), X_train_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'), X_val_num)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'), X_train_city)
np.save(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'), X_val_city)

print("Wszystkie przetworzone zbiory danych zostały zapisane.")

# Zwolnienie pamięci
del df_loc, df_offers, df_processed, X, y, X_train, X_val, y_train, y_val, df_filtered
del X_train_text, X_val_text, X_train_num, X_val_num, X_train_city, X_val_city
gc.collect()

print("\n" + "="*80)
print("CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM. ZRESTARTUJ KERNEL.")
print("="*80)

Wszystkie przetworzone zbiory danych zostały zapisane.

CZĘŚĆ 1 ZAKOŃCZONA POWODZENIEM. ZRESTARTUJ KERNEL.


# Część 2: Budowa, Trening i Inferencia Modelu

In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import gc
import scipy.sparse
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import register_keras_serializable

# Ponowna definicja stałych
ARTIFACTS_DIR = 'artifacts_v12'
PROCESSED_DATA_DIR = 'processed_data_v12'
RANDOM_STATE = 42
MAX_TEXT_FEATURES = 20000
MODEL_PATH = os.path.join(ARTIFACTS_DIR, 'best_location_model_v12.keras')

In [2]:
# Wczytanie artefaktów
print("Wczytywanie artefaktów...")
hierarchy_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'hierarchy_map.joblib'))
id_to_name = joblib.load(os.path.join(ARTIFACTS_DIR, 'id_to_name.joblib'))

# Wczytanie przetworzonych danych
print("Wczytywanie przetworzonych danych...")
X_train_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_train_text.npz'))
X_val_text = scipy.sparse.load_npz(os.path.join(PROCESSED_DATA_DIR, 'X_val_text.npz'))
X_train_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_num.npy'))
X_val_num = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_num.npy'))
X_train_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_train_city.npy'))
X_val_city = np.load(os.path.join(PROCESSED_DATA_DIR, 'X_val_city.npy'))

# Wczytanie ramek danych z etykietami
y_train = pd.read_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_train.pkl'))
y_val = pd.read_pickle(os.path.join(PROCESSED_DATA_DIR, 'y_val.pkl'))
print("Dane wczytane.")

Wczytywanie artefaktów...
Wczytywanie przetworzonych danych...
Dane wczytane.


## Udoskonalona Hierarchiczna Funkcja Straty (Rekomendacja #1)
**Zmiana:** Udoskonalona logika karania. Kara jest stosowana, gdy predykcja jest błędna ORAZ jej przewidywany rodzic również jest błędny, co skupia się na najpoważniejszych pomyłkach.

In [3]:
@register_keras_serializable()
class HierarchicalLoss(tf.keras.losses.Loss):
    def __init__(self, id_to_parent_map, penalty_config, **kwargs):
        super().__init__(**kwargs)
        self.id_to_parent_map = id_to_parent_map
        self.penalty_config = penalty_config
        self.base_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False, reduction='none')
        self.wrong_parent_penalty = tf.constant(penalty_config.get('wrong_parent_penalty', 1.5), dtype=tf.float32)
        
        keys = [int(k) for k in id_to_parent_map.keys()]
        values = [int(v) if pd.notna(v) else 0 for v in id_to_parent_map.values()]
        self.parent_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(keys, values, key_dtype=tf.int64, value_dtype=tf.int64), 0
        )

    def call(self, y_true, y_pred):
        y_true_tensor = tf.cast(y_true, dtype=tf.int64)
        y_pred_ids = tf.cast(tf.argmax(y_pred, axis=-1), dtype=tf.int64)
        base_loss = self.base_loss_fn(y_true_tensor, y_pred)
        
        mask = tf.not_equal(y_true_tensor, 0)
        
        true_parents = self.parent_table.lookup(y_true_tensor)
        pred_parents = self.parent_table.lookup(y_pred_ids)
        
        is_correct = tf.equal(y_true_tensor, y_pred_ids)
        is_parent_correct = tf.equal(true_parents, pred_parents)
        
        # Stosuj karę tylko, gdy predykcja jest błędna ORAZ rodzic też jest błędny
        severe_error_condition = tf.logical_and(tf.logical_not(is_correct), tf.logical_not(is_parent_correct))
        penalties = tf.where(severe_error_condition, self.wrong_parent_penalty, 1.0)
        
        penalized_loss = base_loss * penalties
        masked_loss = tf.where(mask, penalized_loss, 0.0)

        # Uniknięcie dzielenia przez zero, jeśli batch zawiera same zera
        num_valid_labels = tf.reduce_sum(tf.cast(mask, tf.float32))
        return tf.math.divide_no_nan(tf.reduce_sum(masked_loss), num_valid_labels)

    def get_config(self):
        base_config = super().get_config()
        config = {"id_to_parent_map": self.id_to_parent_map, "penalty_config": self.penalty_config}
        return {**base_config, **config}

print("Udoskonalona hierarchiczna funkcja straty zdefiniowana.")

Udoskonalona hierarchiczna funkcja straty zdefiniowana.


In [4]:
def create_mapper(series_list):
    """Tworzy mapowanie z oryginalnych ID na ciągłe indeksy z wielu serii danych."""
    unique_ids = pd.unique(pd.concat(series_list))
    mapper = {val: i for i, val in enumerate(unique_ids)}
    return mapper

def map_labels(series, mapper):
    """Mapuje etykiety używając stworzonego mapowania."""
    return series.map(mapper).fillna(0).astype(int).values

# Mapowanie dla każdej głowicy
district_map = create_mapper([y_train['target_district_id'], y_val['target_district_id']])
subdistrict_map = create_mapper([y_train['target_subdistrict_id'], y_val['target_subdistrict_id']])
street_map = create_mapper([y_train['target_street_id'], y_val['target_street_id']])
city_map = create_mapper([pd.Series(X_train_city), pd.Series(X_val_city)])

# Zapisanie map do późniejszego użytku
joblib.dump(district_map, os.path.join(ARTIFACTS_DIR, 'district_map.joblib'))
joblib.dump(subdistrict_map, os.path.join(ARTIFACTS_DIR, 'subdistrict_map.joblib'))
joblib.dump(street_map, os.path.join(ARTIFACTS_DIR, 'street_map.joblib'))
joblib.dump(city_map, os.path.join(ARTIFACTS_DIR, 'city_map.joblib'))

y_district_train_mapped = map_labels(y_train['target_district_id'], district_map)
y_subdistrict_train_mapped = map_labels(y_train['target_subdistrict_id'], subdistrict_map)
y_street_train_mapped = map_labels(y_train['target_street_id'], street_map)

y_district_val_mapped = map_labels(y_val['target_district_id'], district_map)
y_subdistrict_val_mapped = map_labels(y_val['target_subdistrict_id'], subdistrict_map)
y_street_val_mapped = map_labels(y_val['target_street_id'], street_map)

X_train_city_mapped = map_labels(pd.Series(X_train_city), city_map)
X_val_city_mapped = map_labels(pd.Series(X_val_city), city_map)

# Definicja modelu (architektura bez zmian)
NUM_DISTRICTS = len(district_map)
NUM_SUBDISTRICTS = len(subdistrict_map)
NUM_STREETS = len(street_map)
NUM_CITIES = len(city_map)
NUM_FEATURES = X_train_num.shape[1]

input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True)
input_num = Input(shape=(NUM_FEATURES,), name='num_input')
input_city = Input(shape=(1,), name='city_input')

text_branch = Dense(128, activation='relu')(input_text); text_branch = Dropout(0.3)(text_branch)
num_branch = Dense(64, activation='relu')(input_num); num_branch = Dense(32, activation='relu')(num_branch)
city_branch = Embedding(input_dim=NUM_CITIES, output_dim=50, name='city_embedding')(input_city); city_branch = Flatten()(city_branch)
combined = Concatenate()([text_branch, num_branch, city_branch])
z = Dense(512, activation='relu')(combined); z = Dropout(0.5)(z)
z = Dense(256, activation='relu')(z); z = Dropout(0.5)(z)

output_district = Dense(NUM_DISTRICTS, activation='softmax', name='district_output')(z)
output_subdistrict = Dense(NUM_SUBDISTRICTS, activation='softmax', name='subdistrict_output')(z)
output_street = Dense(NUM_STREETS, activation='softmax', name='street_output')(z)

model = Model(inputs=[input_text, input_num, input_city], outputs=[output_district, output_subdistrict, output_street])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 20000)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 3)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │       2,560,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             256 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 50)             │         276,800 │ city_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 210)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0],             │
│                               │                           │                 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 512)               │         108,032 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 512)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 256)               │         131,328 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_2 (Dropout)           │ (None, 256)               │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 38)                │           9,766 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ subdistrict_output (Dense)    │ (None, 2167)              │         556,919 │ dropout_2[0][0]            │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 12,281,537 (46.85 MB)

 Trainable params: 12,281,537 (46.85 MB)

 Non-trainable params: 0 (0.00 B)

## Trening z Ważeniem Próbek (Rekomendacja #4)
**Zmiana:** Do `model.fit` dodajemy `sample_weight`. Próbki z bardziej szczegółowymi danymi (ulica, pod-dzielnica) otrzymują wyższą wagę, co skłania model do lepszego uczenia się tych trudniejszych przypadków.

In [8]:
# KOMÓRKA TRENINGU - OSTATECZNA POPRAWIONA WERSJA v12.2

# Instancje funkcji straty dla każdej głowicy
penalty_config = {'wrong_parent_penalty': 1.5}
district_loss = 'sparse_categorical_crossentropy'
subdistrict_loss = HierarchicalLoss(hierarchy_map, penalty_config, name='subdistrict_loss')
street_loss = HierarchicalLoss(hierarchy_map, penalty_config, name='street_loss')

model.compile(
    optimizer='adam',
    loss={
        'district_output': district_loss,
        'subdistrict_output': subdistrict_loss,
        'street_output': street_loss
    },
    loss_weights={'district_output': 1.0, 'subdistrict_output': 1.2, 'street_output': 1.5},
    metrics={'district_output': 'accuracy', 'subdistrict_output': 'accuracy', 'street_output': 'accuracy'}
)

# --- OSTATECZNA POPRAWKA: Użycie formatu LISTY dla Y i SAMPLE_WEIGHT ---
# Tworzymy wagi dla poszczególnych wyjść
subdistrict_train_weights = np.where(y_train['target_subdistrict_id'].values > 0, 1.5, 1.0)
street_train_weights = np.where(y_train['target_street_id'].values > 0, 2.0, 1.0)

# Tworzymy LISTĘ wag w tej samej kolejności co wyjścia modelu
sample_weights_list = [
    np.ones(len(y_train)),           # Wagi dla district_output
    subdistrict_train_weights,       # Wagi dla subdistrict_output
    street_train_weights             # Wagi dla street_output
]

# Dane wejściowe pozostają w formacie słownika
X_train_dict = {'text_input': X_train_text, 'num_input': X_train_num, 'city_input': X_train_city_mapped}
X_val_dict = {'text_input': X_val_text, 'num_input': X_val_num, 'city_input': X_val_city_mapped}

# Tworzymy LISTY etykiet (Y) w tej samej kolejności co wyjścia modelu
y_train_list = [
    y_district_train_mapped,
    y_subdistrict_train_mapped,
    y_street_train_mapped
]
y_val_list = [
    y_district_val_mapped,
    y_subdistrict_val_mapped,
    y_street_val_mapped
]
# --- KONIEC POPRAWKI ---

callbacks = [
    ModelCheckpoint(MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
]


history = model.fit(
    X_train_dict, 
    y_train_list, # Przekazujemy listę etykiet
    sample_weight=sample_weights_list, # Przekazujemy listę wag
    validation_data=(X_val_dict, y_val_list), # Zbiór walidacyjny również jako listy
    epochs=20, 
    batch_size=128,
    callbacks=callbacks
)

Epoch 1/20
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - district_output_accuracy: 0.7307 - district_output_loss: 0.8933 - loss: 17.8053 - street_output_accuracy: 0.6368 - street_output_loss: 7.5177 - subdistrict_output_accuracy: 0.5283 - subdistrict_output_loss: 4.6962
Epoch 1: val_loss improved from inf to 9.04966, saving model to artifacts_v12\best_location_model_v12.keras
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 713s 102ms/step - district_output_accuracy: 0.7307 - district_output_loss: 0.8933 - loss: 17.8049 - street_output_accuracy: 0.6368 - street_output_loss: 7.5176 - subdistrict_output_accuracy: 0.5283 - subdistrict_output_loss: 4.6961 - val_district_output_accuracy: 0.8521 - val_district_output_loss: 0.4457 - val_loss: 9.0497 - val_street_output_accuracy: 0.6492 - val_street_output_loss: 3.9319 - val_subdistrict_output_accuracy: 0.6019 - val_subdistrict_output_loss: 2.2553
Epoch 2/20
6935/6935 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - district_output_accuracy: 0.8480 - district_output_loss: 0

## Predykcja Top-K z Post-processingiem (Rekomendacja #2)
**Zmiana:** Zamiast `argmax` używamy `Top-K` do znalezienia kilku najlepszych kandydatów, a następnie szukamy pierwszej spójnej hierarchicznie ścieżki. Zwiększa to odporność na drobne błędy w prawdopodobieństwach.

In [9]:
print(f"Wczytywanie najlepszego modelu z: {MODEL_PATH}")
best_model = tf.keras.models.load_model(
    MODEL_PATH, 
    custom_objects={'HierarchicalLoss': HierarchicalLoss}
)
print("Model wczytany pomyślnie.")

# Wczytujemy mapy odwrotne
inv_district_map = {v: k for k, v in district_map.items()}
inv_subdistrict_map = {v: k for k, v in subdistrict_map.items()}
inv_street_map = {v: k for k, v in street_map.items()}

def find_best_consistent_path(top_k_districts, top_k_subdistricts, top_k_streets, hierarchy_map):
    """Przeszukuje kandydatów Top-K w celu znalezienia pierwszej spójnej ścieżki."""
    for dist_id in top_k_districts:
        # Opcja 1: Sprawdzamy pod-dzielnice pasujące do dzielnicy
        for sub_id in top_k_subdistricts:
            # Warunek 1: Pod-dzielnica musi należeć do dzielnicy (lub jest to 'Brak')
            if sub_id != 0 and hierarchy_map.get(sub_id) != dist_id:
                continue
            
            # Warunek 2: Szukamy ulicy pasującej do pod-dzielnicy (lub do dzielnicy, jeśli pod-dzielnica to 'Brak')
            for street_id in top_k_streets:
                if street_id != 0:
                    expected_parent = sub_id if sub_id != 0 else dist_id
                    if hierarchy_map.get(street_id) != expected_parent:
                        continue
                
                # Znaleziono w pełni spójną ścieżkę
                return dist_id, sub_id, street_id
    
    # Jeśli nie znaleziono spójnej ścieżki, zwróć najbardziej prawdopodobną (argmax)
    return top_k_districts[0], top_k_subdistricts[0], top_k_streets[0]

num_samples = 1000
k = 3 # Liczba kandydatów do sprawdzenia
indices = np.random.choice(range(len(y_val)), num_samples, replace=False)
results_data = []

print(f"Przeprowadzanie predykcji Top-{k} z post-processingiem na {num_samples} próbkach...")
for i in indices:
    dense_vector_sample = X_val_text[i].toarray()
    input_sample = {
        'text_input': dense_vector_sample,
        'num_input': np.expand_dims(X_val_num[i], axis=0),
        'city_input': np.expand_dims(map_labels(pd.Series(X_val_city[i]), city_map), axis=0)
    }
    
    pred_district_probs, pred_subdistrict_probs, pred_street_probs = best_model.predict(input_sample, verbose=0)

    # Pobranie Top-K ID dla każdego poziomu
    top_k_district_ids = [inv_district_map.get(idx, 0) for idx in np.argsort(pred_district_probs[0])[::-1][:k]]
    top_k_subdistrict_ids = [inv_subdistrict_map.get(idx, 0) for idx in np.argsort(pred_subdistrict_probs[0])[::-1][:k]]
    top_k_street_ids = [inv_street_map.get(idx, 0) for idx in np.argsort(pred_street_probs[0])[::-1][:k]]

    # Znalezienie najlepszej spójnej ścieżki
    final_district, final_subdistrict, final_street = find_best_consistent_path(
        top_k_district_ids, top_k_subdistrict_ids, top_k_street_ids, hierarchy_map
    )
    
    true_vals = y_val.iloc[i]
    results_data.append({
        'True_District': true_vals['target_district_id'], 'Pred_District': final_district,
        'True_SubDistrict': true_vals['target_subdistrict_id'], 'Pred_SubDistrict': final_subdistrict,
        'True_Street': true_vals['target_street_id'], 'Pred_Street': final_street
    })

df_results = pd.DataFrame(results_data)

df_results['Correct_District'] = (df_results['True_District'] == df_results['Pred_District'])
df_results['Correct_SubDistrict'] = (df_results['True_SubDistrict'] == df_results['Pred_SubDistrict'])
df_results['Correct_Street'] = (df_results['True_Street'] == df_results['Pred_Street'])
df_results['Correct_Overall'] = df_results['Correct_District'] & df_results['Correct_SubDistrict'] & df_results['Correct_Street']

acc_district = df_results['Correct_District'].mean()
acc_subdistrict = df_results['Correct_SubDistrict'].mean()
acc_street = df_results['Correct_Street'].mean()
acc_overall = df_results['Correct_Overall'].mean()

print("\n--- Wyniki Ewaluacji (po Top-K i Post-processingu) ---")
print(f"Dokładność dla Dzielnic: {acc_district:.2%}")
print(f"Dokładność dla Pod-dzielnic: {acc_subdistrict:.2%}")
print(f"Dokładność dla Ulic: {acc_street:.2%}")
print(f"\nDokładność CAŁKOWITA (wszystkie poziomy poprawne): {acc_overall:.2%}")

Wczytywanie najlepszego modelu z: artifacts_v12\best_location_model_v12.keras
Model wczytany pomyślnie.
Przeprowadzanie predykcji Top-3 z post-processingiem na 1000 próbkach...

--- Wyniki Ewaluacji (po Top-K i Post-processingu) ---
Dokładność dla Dzielnic: 90.20%
Dokładność dla Pod-dzielnic: 64.70%
Dokładność dla Ulic: 67.70%

Dokładność CAŁKOWITA (wszystkie poziomy poprawne): 48.50%


## Zaawansowana Analiza Błędów (Rekomendacja #3)
**Zmiana:** Analizujemy przypadki, w których model popełnił błąd, aby zidentyfikować najczęstsze pomyłki i zrozumieć jego słabości.

In [10]:
for col in ['True_District', 'Pred_District', 'True_SubDistrict', 'Pred_SubDistrict', 'True_Street', 'Pred_Street']:
    df_results[f'{col}_Name'] = df_results[col].apply(lambda x: id_to_name.get(x, 'Brak'))

df_errors = df_results[~df_results['Correct_Overall']].copy()

print("\n--- Analiza Błędów ---")

if not df_errors.empty:
    # Błędy na poziomie dzielnicy
    district_errors = df_errors[~df_errors['Correct_District']]
    if not district_errors.empty:
        print("\nTop 10 najczęstszych pomyłek na poziomie DZIELNICY:")
        district_confusion = district_errors.groupby(['True_District_Name', 'Pred_District_Name']).size().nlargest(10)
        display(district_confusion)
    else:
        print("\nBrak błędów na poziomie DZIELNICY!")
        
    # Błędy na poziomie pod-dzielnicy (przy poprawnej dzielnicy)
    subdistrict_errors = df_errors[df_errors['Correct_District'] & ~df_errors['Correct_SubDistrict']]
    if not subdistrict_errors.empty:
        print("\nTop 10 najczęstszych pomyłek na poziomie POD-DZIELNICY (przy poprawnej dzielnicy):")
        subdistrict_confusion = subdistrict_errors.groupby(['True_SubDistrict_Name', 'Pred_SubDistrict_Name']).size().nlargest(10)
        display(subdistrict_confusion)
    else:
        print("\nBrak błędów na poziomie POD-DZIELNICY przy poprawnych dzielnicach.")
else:
    print("\nModel nie popełnił żadnych błędów! Perfekcyjna dokładność!")

# Wizualizacja losowych wyników
def highlight_correct(s):
    return ['background-color: #d4edda' if s.Correct_Overall else 'background-color: #f8d7da'] * len(s)

print("\n--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---")
styled_results = df_results.sample(min(20, len(df_results)), random_state=RANDOM_STATE).style.apply(highlight_correct, axis=1)
display(styled_results)


--- Analiza Błędów ---

Top 10 najczęstszych pomyłek na poziomie DZIELNICY:


True_District_Name    Pred_District_Name 
Brak                  Kraków-podgórze        5
Wrocław-fabryczna     Wrocław-krzyki         5
Ursynów               Mokotów                4
Brak                  Kraków-nowa huta       3
Wrocław-stare miasto  Wrocław-fabryczna      3
Łódź-górna            Łódź-bałuty            3
Łódź-polesie          Łódź-śródmieście       3
Łódź-widzew           Łódź-bałuty            3
                      Łódź-śródmieście       3
Brak                  Wrocław-śródmieście    2
dtype: int64


Top 10 najczęstszych pomyłek na poziomie POD-DZIELNICY (przy poprawnej dzielnicy):


True_SubDistrict_Name  Pred_SubDistrict_Name
Śródmieście            Brak                     14
Centrum                Brak                      8
Fordon                 Brak                      3
Klimontów              Brak                      3
Ołtaszyn               Krzyki                    3
Azory                  Krowodrza                 2
Brynów                 Brak                      2
Chwaliszewo            Centrum                   2
Godula                 Brak                      2
Grzegórzki             Stare miasto              2
dtype: int64


--- Losowe wyniki predykcji (Zielony = Poprawna, Czerwony = Błędna) ---


,True_District,Pred_District,True_SubDistrict,Pred_SubDistrict,True_Street,Pred_Street,Correct_District,Correct_SubDistrict,Correct_Street,Correct_Overall,True_District_Name,Pred_District_Name,True_SubDistrict_Name,Pred_SubDistrict_Name,True_Street_Name,Pred_Street_Name
521,0,0,71789,0,525878,0,True,False,False,False,Brak,Brak,Stawki,Brak,Okólna,Brak
737,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
740,3255,3255,67365,96066,441095,0,True,False,False,False,Kraków-śródmieście,Kraków-śródmieście,Śródmieście,Prądnik czerwony,Zaułek wileński,Brak
660,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
411,3282,3272,0,84583,270554,0,False,False,False,False,Wrocław-stare miasto,Wrocław-krzyki,Brak,Krzyki,Gwarna,Brak
678,0,0,105466,0,0,0,True,False,True,False,Brak,Brak,Śródmieście,Brak,Brak,Brak
626,3265,3251,69149,0,403906,0,False,False,False,False,Łódź-bałuty,Łódź-widzew,Bałuty,Brak,Sprawiedliwa,Brak
513,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
859,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
136,0,0,0,0,0,0,True,True,True,True,Brak,Brak,Brak,Brak,Brak,Brak
